# Keyword Spotting Dataset Curation

[![Open In Colab <](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ShawnHymel/ei-keyword-spotting/blob/master/ei-audio-dataset-curation.ipynb)

Use this tool to download the Google Speech Commands Dataset, combine it with your own keywords, mix in some background noise, and upload the curated dataset to Edge Impulse. From there, you can train a neural network to classify spoken words and upload it to a microcontroller to perform real-time keyword spotting.

 1. Upload samples of your own keyword (optional)
 2. Adjust parameters in the Settings cell (you will need an [Edge Impulse](https://www.edgeimpulse.com/) account)
 3. Run the rest of the cells! ('shift' + 'enter' on each cell)



### Upload your own keyword samples
You are welcome to use my [custom keyword dataset](https://github.com/ShawnHymel/custom-speech-commands-dataset), but note that it's limited and that I can't promise it will work well. If you want to use it, uncomment the `###Download custom dataset` cell below. You may also add your own recorded keywords to the extracted folder (`/content/custom_keywords`) to augment what's already there.

If you'd rather upload your own custom keyword dataset, follow these instructions:

On the left pane, in the file browser, create a directory structure containing space for your keyword audio samples. All samples for each keyword should be in a directory with that keyword's name.

The audio samples should be `.wav` format, mono, and 1 second long. Bitrate and bitdepth should not matter. Samples shorter than 1 second will be padded with 0s, and samples longer than 1 second will be truncated to 1 second. The exact name of each `.wav` file does not matter, as they will be read, mixed with background noise, and saved to a separate file with an auto-generated name. Directory name does matter (it is used to determine the name of the class during neural network training).

Right-click on each keyword directory and upload all of your samples. Your directory structor should look like the following:

```
/
|- content
|--- custom_keywords
|----- keyword_1
|------- 000.wav
|------- 001.wav
|------- ...
|----- keyword_2
|------- 000.wav
|------- 001.wav
|------- ...
|----- ...
```




In [1]:
### Update Node.js to the latest stable version
!npm cache clean -f
!npm install -g n
!n 20.13.1

npm warn using --force Recommended protections disabled.
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠙⠹⠸⠼
added 1 package in 830ms
⠼  installing : node-v20.13.1
       mkdir : /usr/local/n/versions/node/20.13.1
       fetch : https://nodejs.org/dist/v20.13.1/node-v20.13.1-linux-x64.tar.xz
######################################################################## 100.0%
     copying : node/20.13.1
   installed : v20.13.1 (with npm 10.5.2)

Note: the node command changed location and the old location may be remembered in your current shell.
         old : /tools/node/bin/node
         new : /usr/local/bin/node
If "node --version" shows the old version then start a new shell, or reset the location hash with:
hash -r  (for bash, zsh, ash, dash, and ksh)
rehash   (for csh and tcsh)


In [2]:
### Install required packages and tools
!python -m pip install soundfile
!npm install -g --unsafe-perm edge-impulse-cli@1.26.0 --force

npm WARN using --force Recommended protections disabled.
npm WARN deprecated are-we-there-yet@1.1.7: This package is no longer supported.
npm WARN deprecated npmlog@4.1.2: This package is no longer supported.
npm WARN deprecated debug@4.1.1: Debug versions >=3.2.0 <3.2.7 || >=4 <4.3.1 have a low-severity ReDos regression when used in a Node.js environment. It is recommended you upgrade to 3.2.7 or 4.3.1. (https://github.com/visionmedia/debug/issues/797)
npm WARN deprecated har-validator@5.1.5: this library is no longer supported
npm WARN deprecated osenv@0.1.5: This package is no longer supported.
npm WARN deprecated copy-concurrently@1.0.5: This package is no longer supported.
npm WARN deprecated inflight@1.0.6: This module is not supported, and leaks memory. Do not use it. Check out lru-cache if you want a good and tested way to coalesce async requests by a key value, which is much more comprehensive and powerful.
npm WARN deprecated move-concurrently@1.0.1: This package is no longer

In [4]:
### Settings (You probably do not need to change these)
BASE_DIR = "/content"
OUT_DIR = "keywords_curated"
GOOGLE_DATASET_FILENAME = "speech_commands_v0.02.tar.gz"
GOOGLE_DATASET_URL = "http://download.tensorflow.org/data/" + GOOGLE_DATASET_FILENAME
GOOGLE_DATASET_DIR = "google_speech_commands"
CUSTOM_KEYWORDS_FILENAME = "main.zip"
CUSTOM_KEYWORDS_URL = "https://github.com/ShawnHymel/custom-speech-commands-dataset/archive/" + CUSTOM_KEYWORDS_FILENAME
CUSTOM_KEYWORDS_DIR = "custom_keywords"
CUSTOM_KEYWORDS_REPO_NAME = "custom-speech-commands-dataset-main"
CURATION_SCRIPT = "dataset-curation.py"
CURATION_SCRIPT_URL = "https://raw.githubusercontent.com/ShawnHymel/ei-keyword-spotting/master/" + CURATION_SCRIPT
UTILS_SCRIPT_URL = "https://raw.githubusercontent.com/ShawnHymel/ei-keyword-spotting/master/utils.py"
NUM_SAMPLES = 1500    # Target number of samples to mix and send to Edge Impulse
WORD_VOL = 1.0        # Relative volume of word in output sample
BG_VOL = 0.1          # Relative volume of noise in output sample
SAMPLE_TIME = 1.0     # Time (seconds) of output sample
SAMPLE_RATE = 16000   # Sample rate (Hz) of output sample
BIT_DEPTH = "PCM_16"  # Options: [PCM_16, PCM_24, PCM_32, PCM_U8, FLOAT, DOUBLE]
BG_DIR = "_background_noise_"
TEST_RATIO = 0.2      # 20% reserved for test set, rest is for training
EI_INGEST_TEST_URL = "https://ingestion.edgeimpulse.com/api/test/data"
EI_INGEST_TRAIN_URL = "https://ingestion.edgeimpulse.com/api/training/data"

In [5]:
### Download Google Speech Commands Dataset
!cd {BASE_DIR}
!wget {GOOGLE_DATASET_URL}
!mkdir {GOOGLE_DATASET_DIR}
!echo "Extracting..."
!tar xfz {GOOGLE_DATASET_FILENAME} -C {GOOGLE_DATASET_DIR}

--2025-09-25 19:55:49--  http://download.tensorflow.org/data/speech_commands_v0.02.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.183.207, 209.85.200.207, 64.233.179.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.183.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2428923189 (2.3G) [application/gzip]
Saving to: ‘speech_commands_v0.02.tar.gz’

speech_commands_v0. 100%[===================>]   2.26G   147MB/s    in 17s     

2025-09-25 19:56:05 (138 MB/s) - ‘speech_commands_v0.02.tar.gz’ saved [2428923189/2428923189]

Extracting...


In [6]:
### Pull out background noise directory
!cd {BASE_DIR}
!mv "{GOOGLE_DATASET_DIR}/{BG_DIR}" "{BG_DIR}"

In [ ]:

### (Optional) Download custom dataset--uncomment the code in this cell if you want to use my custom datase

## Download, extract, and move dataset to separate directory
# !cd {BASE_DIR}
# !wget {CUSTOM_KEYWORDS_URL}
# !echo "Extracting..."
# !unzip -q {CUSTOM_KEYWORDS_FILENAME}
# !mv "{CUSTOM_KEYWORDS_REPO_NAME}/{CUSTOM_KEYWORDS_DIR}" "{CUSTOM_KEYWORDS_DIR}"

In [7]:
### User Settings (do change these)

# Location of your custom keyword samples (e.g. "/content/custom_keywords")
# Leave blank ("") for no custom keywords. set to the CUSTOM_KEYWORDS_DIR
# variable to use samples from my custom-speech-commands-dataset repo.
CUSTOM_DATASET_PATH = "/content/custom_keywords"

# Edge Impulse > your_project > Dashboard > Keys
EI_API_KEY = "ei_fce106753156002bdf6daa5750260b24dd81bc60fee64394d66c7c1e575170b9"

# Comma separated words. Must match directory names (that contain samples).
# Recommended: use 2 keywords for microcontroller demo
TARGETS = "hello, stop"

In [8]:
### Download curation and utils scripts
!wget {CURATION_SCRIPT_URL}
!wget {UTILS_SCRIPT_URL}

--2025-09-25 20:21:16--  https://raw.githubusercontent.com/ShawnHymel/ei-keyword-spotting/master/dataset-curation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17427 (17K) [text/plain]
Saving to: ‘dataset-curation.py’

dataset-curation.py 100%[===================>]  17.02K  --.-KB/s    in 0s      

2025-09-25 20:21:16 (53.2 MB/s) - ‘dataset-curation.py’ saved [17427/17427]

--2025-09-25 20:21:16--  https://raw.githubusercontent.com/ShawnHymel/ei-keyword-spotting/master/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [9]:
### Perform curation and mixing of samples with background noise
!cd {BASE_DIR}
!python {CURATION_SCRIPT} \
  -t "{TARGETS}" \
  -n {NUM_SAMPLES} \
  -w {WORD_VOL} \
  -g {BG_VOL} \
  -s {SAMPLE_TIME} \
  -r {SAMPLE_RATE} \
  -e {BIT_DEPTH} \
  -b "{BG_DIR}" \
  -o "{OUT_DIR}" \
  "{GOOGLE_DATASET_DIR}" \
  "{CUSTOM_DATASET_PATH}"

-----------------------------------------------------------------------
Keyword Dataset Curation Tool
v0.1
-----------------------------------------------------------------------
Gathering random background noise snippets (1500 files)
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Mixing: hello (1500 files)
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Mixing: stop (1500 files)
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Mixing: _unknown (1500 files)
Progress: |██████████████████████████████████████████████████| 100.0% Complete
Done!


In [10]:
### Use CLI tool to send curated dataset to Edge Impulse

!cd {BASE_DIR}

# Imports
import os
import random

# Seed with system time
random.seed()

# Go through each category in our curated dataset
for dir in os.listdir(OUT_DIR):

  # Create list of files for one category
  paths = []
  for filename in os.listdir(os.path.join(OUT_DIR, dir)):
    paths.append(os.path.join(OUT_DIR, dir, filename))

  # Shuffle and divide into test and training sets
  random.shuffle(paths)
  num_test_samples = int(TEST_RATIO * len(paths))
  test_paths = paths[:num_test_samples]
  train_paths = paths[num_test_samples:]

  # Create arugments list (as a string) for CLI call
  test_paths = ['"' + s + '"' for s in test_paths]
  test_paths = ' '.join(test_paths)
  train_paths = ['"' + s + '"' for s in train_paths]
  train_paths = ' '.join(train_paths)

  # Send test files to Edge Impulse
  !edge-impulse-uploader \
    --category testing \
    --label {dir} \
    --api-key {EI_API_KEY} \
    --silent \
    {test_paths}

  # # Send training files to Edge Impulse
  !edge-impulse-uploader \
    --category training \
    --label {dir} \
    --api-key {EI_API_KEY} \
    --silent \
    {train_paths}

Streaming output truncated to the last 5000 lines.
[ 701/1200] Uploading keywords_curated/_noise/_noise.1165.wav OK (476 ms)
[ 702/1200] Uploading keywords_curated/_noise/_noise.1339.wav OK (357 ms)
[ 703/1200] Uploading keywords_curated/_noise/_noise.0377.wav OK (277 ms)
[ 704/1200] Uploading keywords_curated/_noise/_noise.0221.wav OK (407 ms)
[ 705/1200] Uploading keywords_curated/_noise/_noise.0630.wav OK (417 ms)
[ 706/1200] Uploading keywords_curated/_noise/_noise.0674.wav OK (424 ms)
[ 707/1200] Uploading keywords_curated/_noise/_noise.0853.wav OK (319 ms)
[ 708/1200] Uploading keywords_curated/_noise/_noise.0393.wav OK (297 ms)
[ 709/1200] Uploading keywords_curated/_noise/_noise.0175.wav OK (427 ms)
[ 710/1200] Uploading keywords_curated/_noise/_noise.1184.wav OK (331 ms)
[ 711/1200] Uploading keywords_curated/_noise/_noise.0604.wav OK (361 ms)
[ 712/1200] Uploading keywords_curated/_noise/_noise.0974.wav OK (263 ms)
[ 713/1200] Uploading keywords_curated/_noise/_noise.1096.wav